# Comprehensive Regression Analysis for Football Analytics Project

This notebook implements regression models for all three research questions:
1. **Question 1**: Possession vs Match Outcomes (Logistic Regression)
2. **Question 2**: Salary vs Performance (Multiple Linear Regression)
3. **Question 3**: Age vs Performance (Linear and Polynomial Regression)

Each section includes:
- Model building and training
- Model evaluation (R², RMSE, MAE, classification metrics)
- Visualization of predictions vs actual
- Feature importance/coefficients analysis
- Statistical significance testing

## Import Libraries

In [ ]:
# Data manipulation and analysis
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Machine Learning
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import LogisticRegression, LinearRegression, Ridge, Lasso
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, confusion_matrix,
    classification_report, roc_auc_score, roc_curve,
    r2_score, mean_squared_error, mean_absolute_error
)

# Statistical analysis
from scipy import stats
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Settings
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
np.random.seed(42)

print("Libraries imported successfully!")

---
# Question 1: Possession vs Match Outcomes - Logistic Regression

**Research Question**: Can possession percentage predict match results?

**Approach**: 
- Multinomial Logistic Regression (3 classes: Win, Draw, Loss)
- Binary Logistic Regression (Win vs Not-Win)

## 1.1 Load and Prepare Data

In [ ]:
# Load possession data
possession_df = pd.read_csv('../data_raw/PL_LaLiga_possession_2023_2024.csv')

# Display basic info
print("Dataset Shape:", possession_df.shape)
print("\nFirst few rows:")
print(possession_df.head())
print("\nData types:")
print(possession_df.dtypes)
print("\nMissing values:")
print(possession_df.isnull().sum())

In [ ]:
# Prepare features and target
# Feature: Possession percentage
X_poss = possession_df[['Poss']].values

# Target: Match result
# Map results to numeric: W=2, D=1, L=0
result_mapping = {'W': 2, 'D': 1, 'L': 0}
y_poss_multi = possession_df['Result'].map(result_mapping).values

# Binary target: Win vs Not-Win
y_poss_binary = (possession_df['Result'] == 'W').astype(int).values

print("Feature shape:", X_poss.shape)
print("Target (multiclass) shape:", y_poss_multi.shape)
print("Target (binary) shape:", y_poss_binary.shape)
print("\nClass distribution (multiclass):")
print(pd.Series(y_poss_multi).value_counts().sort_index())
print("\nClass distribution (binary):")
print(pd.Series(y_poss_binary).value_counts())

## 1.2 Binary Logistic Regression: Win vs Not-Win

In [ ]:
# Split data
X_train_poss, X_test_poss, y_train_binary, y_test_binary = train_test_split(
    X_poss, y_poss_binary, test_size=0.2, random_state=42, stratify=y_poss_binary
)

# Standardize features
scaler_poss = StandardScaler()
X_train_poss_scaled = scaler_poss.fit_transform(X_train_poss)
X_test_poss_scaled = scaler_poss.transform(X_test_poss)

# Train logistic regression model
log_reg_binary = LogisticRegression(random_state=42, max_iter=1000)
log_reg_binary.fit(X_train_poss_scaled, y_train_binary)

# Predictions
y_pred_binary = log_reg_binary.predict(X_test_poss_scaled)
y_pred_proba_binary = log_reg_binary.predict_proba(X_test_poss_scaled)[:, 1]

print("Binary Logistic Regression Model Trained!")
print(f"\nCoefficient (Possession): {log_reg_binary.coef_[0][0]:.4f}")
print(f"Intercept: {log_reg_binary.intercept_[0]:.4f}")

In [ ]:
# Evaluate binary model
print("=" * 60)
print("BINARY LOGISTIC REGRESSION RESULTS (Win vs Not-Win)")
print("=" * 60)

# Accuracy metrics
accuracy = accuracy_score(y_test_binary, y_pred_binary)
precision = precision_score(y_test_binary, y_pred_binary)
recall = recall_score(y_test_binary, y_pred_binary)
f1 = f1_score(y_test_binary, y_pred_binary)

print(f"\nAccuracy:  {accuracy:.4f} ({accuracy*100:.2f}%)")
print(f"Precision: {precision:.4f}")
print(f"Recall:    {recall:.4f}")
print(f"F1-Score:  {f1:.4f}")

# ROC-AUC
try:
    roc_auc = roc_auc_score(y_test_binary, y_pred_proba_binary)
    print(f"ROC-AUC:   {roc_auc:.4f}")
except:
    print("ROC-AUC:   Could not calculate")

# Confusion Matrix
print("\nConfusion Matrix:")
cm_binary = confusion_matrix(y_test_binary, y_pred_binary)
print(cm_binary)

# Classification Report
print("\nDetailed Classification Report:")
print(classification_report(y_test_binary, y_pred_binary, 
                          target_names=['Not-Win', 'Win']))

In [ ]:
# Visualize binary model results
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# 1. Confusion Matrix Heatmap
sns.heatmap(cm_binary, annot=True, fmt='d', cmap='Blues', ax=axes[0],
            xticklabels=['Not-Win', 'Win'], yticklabels=['Not-Win', 'Win'])
axes[0].set_title('Confusion Matrix - Win vs Not-Win', fontsize=14, fontweight='bold')
axes[0].set_ylabel('Actual', fontsize=12)
axes[0].set_xlabel('Predicted', fontsize=12)

# 2. ROC Curve
try:
    fpr, tpr, thresholds = roc_curve(y_test_binary, y_pred_proba_binary)
    axes[1].plot(fpr, tpr, linewidth=2, label=f'ROC Curve (AUC = {roc_auc:.3f})')
    axes[1].plot([0, 1], [0, 1], 'k--', linewidth=1, label='Random Classifier')
    axes[1].set_xlabel('False Positive Rate', fontsize=12)
    axes[1].set_ylabel('True Positive Rate', fontsize=12)
    axes[1].set_title('ROC Curve', fontsize=14, fontweight='bold')
    axes[1].legend(loc='lower right')
    axes[1].grid(True, alpha=0.3)
except:
    axes[1].text(0.5, 0.5, 'ROC Curve\nNot Available', 
                ha='center', va='center', fontsize=12)

# 3. Possession Distribution by Actual Result
test_results_df = pd.DataFrame({
    'Possession': X_test_poss.flatten(),
    'Actual': ['Win' if y == 1 else 'Not-Win' for y in y_test_binary],
    'Predicted': ['Win' if y == 1 else 'Not-Win' for y in y_pred_binary]
})

sns.boxplot(data=test_results_df, x='Actual', y='Possession', ax=axes[2])
axes[2].set_title('Possession Distribution by Result', fontsize=14, fontweight='bold')
axes[2].set_xlabel('Match Result', fontsize=12)
axes[2].set_ylabel('Possession %', fontsize=12)

plt.tight_layout()
plt.show()

# Statistical summary
print("\nPossession Statistics by Actual Result:")
print(test_results_df.groupby('Actual')['Possession'].describe())

## 1.3 Multinomial Logistic Regression: Win vs Draw vs Loss

In [ ]:
# Split data for multiclass
X_train_poss_m, X_test_poss_m, y_train_multi, y_test_multi = train_test_split(
    X_poss, y_poss_multi, test_size=0.2, random_state=42, stratify=y_poss_multi
)

# Standardize
scaler_poss_m = StandardScaler()
X_train_poss_m_scaled = scaler_poss_m.fit_transform(X_train_poss_m)
X_test_poss_m_scaled = scaler_poss_m.transform(X_test_poss_m)

# Train multinomial logistic regression
log_reg_multi = LogisticRegression(multi_class='multinomial', solver='lbfgs', 
                                   random_state=42, max_iter=1000)
log_reg_multi.fit(X_train_poss_m_scaled, y_train_multi)

# Predictions
y_pred_multi = log_reg_multi.predict(X_test_poss_m_scaled)
y_pred_proba_multi = log_reg_multi.predict_proba(X_test_poss_m_scaled)

print("Multinomial Logistic Regression Model Trained!")
print(f"\nCoefficients for each class:")
for i, class_name in enumerate(['Loss', 'Draw', 'Win']):
    print(f"  {class_name}: {log_reg_multi.coef_[i][0]:.4f}")

In [ ]:
# Evaluate multiclass model
print("=" * 60)
print("MULTINOMIAL LOGISTIC REGRESSION RESULTS (Win vs Draw vs Loss)")
print("=" * 60)

# Accuracy
accuracy_multi = accuracy_score(y_test_multi, y_pred_multi)
print(f"\nOverall Accuracy: {accuracy_multi:.4f} ({accuracy_multi*100:.2f}%)")

# Confusion Matrix
print("\nConfusion Matrix:")
cm_multi = confusion_matrix(y_test_multi, y_pred_multi)
print(cm_multi)

# Classification Report
print("\nDetailed Classification Report:")
print(classification_report(y_test_multi, y_pred_multi, 
                          target_names=['Loss', 'Draw', 'Win']))

# Weighted metrics
precision_weighted = precision_score(y_test_multi, y_pred_multi, average='weighted')
recall_weighted = recall_score(y_test_multi, y_pred_multi, average='weighted')
f1_weighted = f1_score(y_test_multi, y_pred_multi, average='weighted')

print(f"\nWeighted Metrics:")
print(f"  Precision: {precision_weighted:.4f}")
print(f"  Recall:    {recall_weighted:.4f}")
print(f"  F1-Score:  {f1_weighted:.4f}")

In [ ]:
# Visualize multiclass results
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 1. Confusion Matrix
sns.heatmap(cm_multi, annot=True, fmt='d', cmap='YlOrRd', ax=axes[0],
            xticklabels=['Loss', 'Draw', 'Win'], 
            yticklabels=['Loss', 'Draw', 'Win'])
axes[0].set_title('Confusion Matrix - Multiclass', fontsize=14, fontweight='bold')
axes[0].set_ylabel('Actual Result', fontsize=12)
axes[0].set_xlabel('Predicted Result', fontsize=12)

# 2. Possession by Result
test_multi_df = pd.DataFrame({
    'Possession': X_test_poss_m.flatten(),
    'Actual': ['Loss' if y == 0 else 'Draw' if y == 1 else 'Win' for y in y_test_multi]
})

sns.violinplot(data=test_multi_df, x='Actual', y='Possession', ax=axes[1],
              order=['Loss', 'Draw', 'Win'])
axes[1].set_title('Possession Distribution by Match Result', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Match Result', fontsize=12)
axes[1].set_ylabel('Possession %', fontsize=12)

plt.tight_layout()
plt.show()

# Mean possession by result
print("\nMean Possession by Result:")
print(test_multi_df.groupby('Actual')['Possession'].agg(['mean', 'std', 'count']))

## 1.4 Statistical Significance Testing

In [ ]:
# Use statsmodels for detailed statistics
X_poss_with_const = sm.add_constant(X_poss)

# Fit logistic regression using statsmodels for p-values
logit_model = sm.Logit(y_poss_binary, X_poss_with_const)
logit_result = logit_model.fit()

print("=" * 60)
print("STATISTICAL SIGNIFICANCE TEST (Statsmodels)")
print("=" * 60)
print(logit_result.summary())

# Extract key statistics
print("\n" + "=" * 60)
print("KEY FINDINGS")
print("=" * 60)
print(f"Coefficient for Possession: {logit_result.params[1]:.6f}")
print(f"P-value: {logit_result.pvalues[1]:.6f}")
print(f"Odds Ratio: {np.exp(logit_result.params[1]):.6f}")
print(f"\nInterpretation:")
if logit_result.pvalues[1] < 0.05:
    print("  ✓ Possession is STATISTICALLY SIGNIFICANT (p < 0.05)")
    print(f"  ✓ For every 1% increase in possession, odds of winning increase by {(np.exp(logit_result.params[1])-1)*100:.2f}%")
else:
    print("  ✗ Possession is NOT statistically significant (p >= 0.05)")
    print("  ✗ Cannot confidently conclude possession predicts match outcome")

---
# Question 2: Salary vs Performance - Multiple Linear Regression

**Research Question**: Do performance metrics predict player salaries?

**Approach**: Multiple Linear Regression with performance metrics as predictors

## 2.1 Load and Prepare Salary Data

In [ ]:
# Load salary data
salary_df = pd.read_csv('../data_raw/player_stats_with_salaries.csv')

print("Dataset Shape:", salary_df.shape)
print("\nFirst few rows:")
print(salary_df.head())
print("\nColumn names:")
print(salary_df.columns.tolist())
print("\nMissing values:")
print(salary_df.isnull().sum())

In [ ]:
# Clean and prepare salary data
# Select relevant columns for regression
performance_cols = ['Gls', 'Ast', 'xG', 'npxG', 'xAG', 'Min', 'MP']
target_col = 'Annual Gross'

# Check which columns exist
available_perf_cols = [col for col in performance_cols if col in salary_df.columns]
print(f"Available performance columns: {available_perf_cols}")

# Check target column
if target_col in salary_df.columns:
    print(f"Target column '{target_col}' found")
else:
    # Try alternative column names
    salary_cols = [col for col in salary_df.columns if 'salary' in col.lower() or 'annual' in col.lower()]
    print(f"Alternative salary columns: {salary_cols}")
    if salary_cols:
        target_col = salary_cols[0]
        print(f"Using '{target_col}' as target")

# Create clean dataset
regression_cols = available_perf_cols + [target_col]
salary_clean = salary_df[regression_cols].copy()

# Remove rows with missing values
salary_clean = salary_clean.dropna()

print(f"\nClean dataset shape: {salary_clean.shape}")
print(f"Rows dropped due to missing values: {len(salary_df) - len(salary_clean)}")
print("\nDescriptive statistics:")
print(salary_clean.describe())

In [ ]:
# Check for and handle outliers using IQR method
def remove_outliers_iqr(df, columns, multiplier=1.5):
    df_clean = df.copy()
    for col in columns:
        Q1 = df_clean[col].quantile(0.25)
        Q3 = df_clean[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - multiplier * IQR
        upper_bound = Q3 + multiplier * IQR
        
        before = len(df_clean)
        df_clean = df_clean[(df_clean[col] >= lower_bound) & (df_clean[col] <= upper_bound)]
        after = len(df_clean)
        removed = before - after
        if removed > 0:
            print(f"{col}: Removed {removed} outliers ({removed/before*100:.1f}%)")
    return df_clean

# Optional: Remove extreme outliers
print("Outlier analysis:")
salary_no_outliers = remove_outliers_iqr(salary_clean, [target_col], multiplier=3.0)
print(f"\nDataset after outlier removal: {salary_no_outliers.shape}")

# Use dataset without extreme outliers
salary_final = salary_no_outliers.copy()

## 2.2 Prepare Features and Target

In [ ]:
# Separate features and target
X_salary = salary_final[available_perf_cols].values
y_salary = salary_final[target_col].values

# Split into train and test sets
X_train_sal, X_test_sal, y_train_sal, y_test_sal = train_test_split(
    X_salary, y_salary, test_size=0.2, random_state=42
)

# Standardize features
scaler_salary = StandardScaler()
X_train_sal_scaled = scaler_salary.fit_transform(X_train_sal)
X_test_sal_scaled = scaler_salary.transform(X_test_sal)

print(f"Training set: {X_train_sal_scaled.shape}")
print(f"Test set: {X_test_sal_scaled.shape}")
print(f"\nFeature names: {available_perf_cols}")

## 2.3 Check for Multicollinearity

In [ ]:
# Correlation matrix
correlation_matrix = salary_final[available_perf_cols].corr()

plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='coolwarm', 
            square=True, linewidths=0.5, cbar_kws={"shrink": 0.8})
plt.title('Feature Correlation Matrix', fontsize=16, fontweight='bold', pad=20)
plt.tight_layout()
plt.show()

# Variance Inflation Factor (VIF)
print("\nVariance Inflation Factor (VIF):")
print("VIF > 10 indicates high multicollinearity")
print("-" * 40)

vif_data = pd.DataFrame()
vif_data["Feature"] = available_perf_cols
vif_data["VIF"] = [variance_inflation_factor(X_train_sal_scaled, i) 
                   for i in range(len(available_perf_cols))]
vif_data = vif_data.sort_values('VIF', ascending=False)
print(vif_data.to_string(index=False))

# Identify problematic features
high_vif = vif_data[vif_data['VIF'] > 10]
if len(high_vif) > 0:
    print("\n⚠️  High multicollinearity detected in:")
    print(high_vif.to_string(index=False))
else:
    print("\n✓ No severe multicollinearity detected")

## 2.4 Train Multiple Linear Regression Models

In [ ]:
# 1. Ordinary Least Squares (OLS) Linear Regression
lr_model = LinearRegression()
lr_model.fit(X_train_sal_scaled, y_train_sal)
y_pred_lr = lr_model.predict(X_test_sal_scaled)

# 2. Ridge Regression (L2 regularization)
ridge_model = Ridge(alpha=1.0, random_state=42)
ridge_model.fit(X_train_sal_scaled, y_train_sal)
y_pred_ridge = ridge_model.predict(X_test_sal_scaled)

# 3. Lasso Regression (L1 regularization)
lasso_model = Lasso(alpha=1.0, random_state=42, max_iter=10000)
lasso_model.fit(X_train_sal_scaled, y_train_sal)
y_pred_lasso = lasso_model.predict(X_test_sal_scaled)

print("All models trained successfully!")

## 2.5 Model Evaluation and Comparison

In [ ]:
# Calculate metrics for each model
def calculate_metrics(y_true, y_pred, model_name):
    r2 = r2_score(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    
    return {
        'Model': model_name,
        'R²': r2,
        'RMSE': rmse,
        'MAE': mae,
        'MAPE (%)': mape
    }

# Compare models
models_comparison = pd.DataFrame([
    calculate_metrics(y_test_sal, y_pred_lr, 'Linear Regression'),
    calculate_metrics(y_test_sal, y_pred_ridge, 'Ridge Regression'),
    calculate_metrics(y_test_sal, y_pred_lasso, 'Lasso Regression')
])

print("=" * 70)
print("MODEL COMPARISON - SALARY PREDICTION")
print("=" * 70)
print(models_comparison.to_string(index=False))
print("\nBest model by R²:", models_comparison.loc[models_comparison['R²'].idxmax(), 'Model'])
print("Best model by RMSE:", models_comparison.loc[models_comparison['RMSE'].idxmin(), 'Model'])

In [ ]:
# Cross-validation for robust evaluation
print("\n" + "=" * 70)
print("5-FOLD CROSS-VALIDATION RESULTS")
print("=" * 70)

models = {
    'Linear Regression': lr_model,
    'Ridge Regression': ridge_model,
    'Lasso Regression': lasso_model
}

cv_results = []
for name, model in models.items():
    scores = cross_val_score(model, X_train_sal_scaled, y_train_sal, 
                            cv=5, scoring='r2')
    cv_results.append({
        'Model': name,
        'Mean R²': scores.mean(),
        'Std R²': scores.std(),
        'Min R²': scores.min(),
        'Max R²': scores.max()
    })

cv_df = pd.DataFrame(cv_results)
print(cv_df.to_string(index=False))

## 2.6 Feature Importance Analysis

In [ ]:
# Extract and compare coefficients
coef_comparison = pd.DataFrame({
    'Feature': available_perf_cols,
    'Linear Reg': lr_model.coef_,
    'Ridge': ridge_model.coef_,
    'Lasso': lasso_model.coef_
})

print("=" * 70)
print("FEATURE COEFFICIENTS (Standardized)")
print("=" * 70)
print(coef_comparison.to_string(index=False))

# Visualize coefficients
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

for idx, (model_name, col_name) in enumerate([('Linear Regression', 'Linear Reg'),
                                                ('Ridge Regression', 'Ridge'),
                                                ('Lasso Regression', 'Lasso')]):
    coef_df = coef_comparison[['Feature', col_name]].sort_values(col_name, ascending=False)
    
    axes[idx].barh(coef_df['Feature'], coef_df[col_name], 
                   color=['green' if x > 0 else 'red' for x in coef_df[col_name]])
    axes[idx].set_xlabel('Coefficient Value', fontsize=11)
    axes[idx].set_title(f'{model_name}\nFeature Importance', 
                       fontsize=12, fontweight='bold')
    axes[idx].axvline(x=0, color='black', linestyle='--', linewidth=1)
    axes[idx].grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.show()

# Identify most important features
abs_coefs = coef_comparison.copy()
abs_coefs['Avg_Abs_Coef'] = abs_coefs[['Linear Reg', 'Ridge', 'Lasso']].abs().mean(axis=1)
abs_coefs = abs_coefs.sort_values('Avg_Abs_Coef', ascending=False)

print("\nMost Important Features (by average absolute coefficient):")
print(abs_coefs[['Feature', 'Avg_Abs_Coef']].to_string(index=False))

## 2.7 Prediction Visualization

In [ ]:
# Actual vs Predicted plots
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

predictions = [
    ('Linear Regression', y_pred_lr),
    ('Ridge Regression', y_pred_ridge),
    ('Lasso Regression', y_pred_lasso)
]

for idx, (name, y_pred) in enumerate(predictions):
    # Scatter plot
    axes[idx].scatter(y_test_sal, y_pred, alpha=0.6, s=50)
    
    # Perfect prediction line
    min_val = min(y_test_sal.min(), y_pred.min())
    max_val = max(y_test_sal.max(), y_pred.max())
    axes[idx].plot([min_val, max_val], [min_val, max_val], 
                   'r--', linewidth=2, label='Perfect Prediction')
    
    # Labels and title
    axes[idx].set_xlabel('Actual Salary', fontsize=11)
    axes[idx].set_ylabel('Predicted Salary', fontsize=11)
    
    r2 = r2_score(y_test_sal, y_pred)
    axes[idx].set_title(f'{name}\nR² = {r2:.4f}', 
                       fontsize=12, fontweight='bold')
    axes[idx].legend()
    axes[idx].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Residual analysis for best model (typically Linear Regression)
residuals = y_test_sal - y_pred_lr

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# 1. Residuals vs Predicted
axes[0].scatter(y_pred_lr, residuals, alpha=0.6)
axes[0].axhline(y=0, color='r', linestyle='--', linewidth=2)
axes[0].set_xlabel('Predicted Salary', fontsize=11)
axes[0].set_ylabel('Residuals', fontsize=11)
axes[0].set_title('Residual Plot', fontsize=12, fontweight='bold')
axes[0].grid(True, alpha=0.3)

# 2. Histogram of residuals
axes[1].hist(residuals, bins=30, edgecolor='black', alpha=0.7)
axes[1].axvline(x=0, color='r', linestyle='--', linewidth=2)
axes[1].set_xlabel('Residual Value', fontsize=11)
axes[1].set_ylabel('Frequency', fontsize=11)
axes[1].set_title('Distribution of Residuals', fontsize=12, fontweight='bold')
axes[1].grid(True, alpha=0.3)

# 3. Q-Q plot
stats.probplot(residuals, dist="norm", plot=axes[2])
axes[2].set_title('Q-Q Plot (Normality Check)', fontsize=12, fontweight='bold')
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Statistical test for normality
statistic, p_value = stats.shapiro(residuals)
print(f"\nShapiro-Wilk Test for Normality of Residuals:")
print(f"  Statistic: {statistic:.6f}")
print(f"  P-value: {p_value:.6f}")
if p_value > 0.05:
    print("  ✓ Residuals appear normally distributed (p > 0.05)")
else:
    print("  ⚠️  Residuals may not be normally distributed (p < 0.05)")

## 2.8 Statistical Significance with Statsmodels

In [ ]:
# Use statsmodels for p-values and detailed statistics
X_sal_with_const = sm.add_constant(X_train_sal_scaled)
ols_model = sm.OLS(y_train_sal, X_sal_with_const)
ols_result = ols_model.fit()

print("=" * 70)
print("DETAILED STATISTICAL ANALYSIS (OLS Regression Summary)")
print("=" * 70)
print(ols_result.summary())

# Extract coefficient table
print("\n" + "=" * 70)
print("COEFFICIENT SIGNIFICANCE")
print("=" * 70)
coef_table = pd.DataFrame({
    'Feature': ['Intercept'] + available_perf_cols,
    'Coefficient': ols_result.params.values,
    'Std Error': ols_result.bse.values,
    't-statistic': ols_result.tvalues.values,
    'P-value': ols_result.pvalues.values,
    'Significant': ['Yes' if p < 0.05 else 'No' for p in ols_result.pvalues.values]
})
print(coef_table.to_string(index=False))

# Highlight significant predictors
significant_features = coef_table[coef_table['P-value'] < 0.05]['Feature'].tolist()
print(f"\n✓ Statistically significant predictors (p < 0.05):")
for feat in significant_features:
    if feat != 'Intercept':
        print(f"  • {feat}")

---
# Question 3: Age vs Performance - Linear and Polynomial Regression

**Research Question**: How does age affect player performance?

**Approach**: 
- Linear regression to test linear relationship
- Polynomial regression to capture potential inverted-U pattern

## 3.1 Load and Prepare Age-Performance Data

In [ ]:
# Load player stats data
age_df = pd.read_csv('../data_raw/playerStats.csv')

print("Dataset Shape:", age_df.shape)
print("\nFirst few rows:")
print(age_df.head())
print("\nColumn names:")
print(age_df.columns.tolist())
print("\nData types:")
print(age_df.dtypes)

In [ ]:
# Parse age from "YY-DDD" format to just years
def parse_age(age_str):
    try:
        if pd.isna(age_str):
            return np.nan
        # Extract years from format like "25-064"
        return int(str(age_str).split('-')[0])
    except:
        return np.nan

age_df['Age_Years'] = age_df['Age'].apply(parse_age)

# Remove players with missing age
age_df_clean = age_df.dropna(subset=['Age_Years']).copy()

print(f"Original dataset: {len(age_df)} players")
print(f"After removing missing ages: {len(age_df_clean)} players")
print(f"\nAge distribution:")
print(age_df_clean['Age_Years'].describe())

In [ ]:
# Convert performance metrics to numeric
performance_metrics = ['Gls', 'Ast', 'xG', 'npxG', 'xAG', 'Min', 'MP', 'PrgC', 'PrgP']

# Check which columns exist
available_metrics = [col for col in performance_metrics if col in age_df_clean.columns]
print(f"Available metrics: {available_metrics}")

# Convert to numeric
for col in available_metrics:
    age_df_clean[col] = pd.to_numeric(age_df_clean[col], errors='coerce')

# Fill missing minutes with 0 (players who didn't play)
if 'Min' in available_metrics:
    age_df_clean['Min'] = age_df_clean['Min'].fillna(0)

# Create per-90 metrics
print("\nCreating per-90 minute metrics...")
per90_metrics = []

# Only create per-90 metrics for players with > 90 minutes
age_df_clean = age_df_clean[age_df_clean['Min'] > 90].copy()

for metric in ['Gls', 'Ast', 'xG', 'xAG', 'PrgC', 'PrgP']:
    if metric in available_metrics:
        new_col = f'{metric}_Per_90'
        age_df_clean[new_col] = (age_df_clean[metric] / age_df_clean['Min']) * 90
        per90_metrics.append(new_col)
        print(f"  Created: {new_col}")

# Remove any infinite values
age_df_clean = age_df_clean.replace([np.inf, -np.inf], np.nan)
age_df_clean = age_df_clean.dropna(subset=per90_metrics)

print(f"\nFinal dataset for analysis: {len(age_df_clean)} players")
print(f"Per-90 metrics created: {per90_metrics}")

## 3.2 Create Composite Performance Score

In [ ]:
# Create a composite performance score
from sklearn.preprocessing import MinMaxScaler

# Select key offensive metrics for composite score
key_metrics = ['Gls_Per_90', 'Ast_Per_90', 'xG_Per_90']
key_metrics = [m for m in key_metrics if m in age_df_clean.columns]

if len(key_metrics) > 0:
    # Scale each metric to 0-1
    scaler_perf = MinMaxScaler()
    scaled_metrics = scaler_perf.fit_transform(age_df_clean[key_metrics])
    
    # Composite = average of scaled metrics
    age_df_clean['Performance_Score'] = scaled_metrics.mean(axis=1)
    
    print("Composite Performance Score created!")
    print(f"Based on: {key_metrics}")
    print(f"\nPerformance Score statistics:")
    print(age_df_clean['Performance_Score'].describe())
else:
    print("⚠️  Not enough metrics to create composite score")

## 3.3 Exploratory Analysis: Age vs Performance

In [ ]:
# Visualize relationship between age and performance
fig, axes = plt.subplots(2, 3, figsize=(18, 10))
axes = axes.flatten()

# Plot each per-90 metric vs age
plot_metrics = per90_metrics[:5] + ['Performance_Score'] if 'Performance_Score' in age_df_clean.columns else per90_metrics[:6]

for idx, metric in enumerate(plot_metrics):
    if metric in age_df_clean.columns:
        axes[idx].scatter(age_df_clean['Age_Years'], age_df_clean[metric], 
                         alpha=0.3, s=20)
        
        # Add trend line
        z = np.polyfit(age_df_clean['Age_Years'], age_df_clean[metric], 2)
        p = np.poly1d(z)
        x_line = np.linspace(age_df_clean['Age_Years'].min(), 
                            age_df_clean['Age_Years'].max(), 100)
        axes[idx].plot(x_line, p(x_line), 'r-', linewidth=2, label='Polynomial Fit')
        
        axes[idx].set_xlabel('Age', fontsize=11)
        axes[idx].set_ylabel(metric, fontsize=11)
        axes[idx].set_title(f'Age vs {metric}', fontsize=12, fontweight='bold')
        axes[idx].legend()
        axes[idx].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Correlation analysis
print("=" * 70)
print("CORRELATION BETWEEN AGE AND PERFORMANCE METRICS")
print("=" * 70)

correlations = []
for metric in per90_metrics + (['Performance_Score'] if 'Performance_Score' in age_df_clean.columns else []):
    if metric in age_df_clean.columns:
        # Pearson correlation
        pearson_r, pearson_p = stats.pearsonr(age_df_clean['Age_Years'], 
                                             age_df_clean[metric])
        # Spearman correlation
        spearman_r, spearman_p = stats.spearmanr(age_df_clean['Age_Years'], 
                                                age_df_clean[metric])
        
        correlations.append({
            'Metric': metric,
            'Pearson r': pearson_r,
            'Pearson p': pearson_p,
            'Spearman r': spearman_r,
            'Spearman p': spearman_p,
            'Significant': 'Yes' if pearson_p < 0.05 else 'No'
        })

corr_df = pd.DataFrame(correlations)
print(corr_df.to_string(index=False))

print("\n✓ Negative correlation = Performance decreases with age")
print("✓ Positive correlation = Performance increases with age")

## 3.4 Linear Regression: Age vs Performance Score

In [ ]:
# Prepare data for regression
if 'Performance_Score' in age_df_clean.columns:
    X_age = age_df_clean[['Age_Years']].values
    y_perf = age_df_clean['Performance_Score'].values
    
    # Split data
    X_train_age, X_test_age, y_train_perf, y_test_perf = train_test_split(
        X_age, y_perf, test_size=0.2, random_state=42
    )
    
    # Train linear model
    lr_age_model = LinearRegression()
    lr_age_model.fit(X_train_age, y_train_perf)
    y_pred_age_lr = lr_age_model.predict(X_test_age)
    
    # Evaluate
    r2_lr = r2_score(y_test_perf, y_pred_age_lr)
    rmse_lr = np.sqrt(mean_squared_error(y_test_perf, y_pred_age_lr))
    mae_lr = mean_absolute_error(y_test_perf, y_pred_age_lr)
    
    print("=" * 70)
    print("LINEAR REGRESSION: Age vs Performance Score")
    print("=" * 70)
    print(f"Coefficient (slope): {lr_age_model.coef_[0]:.6f}")
    print(f"Intercept: {lr_age_model.intercept_:.6f}")
    print(f"\nModel Performance:")
    print(f"  R² Score:  {r2_lr:.4f}")
    print(f"  RMSE:      {rmse_lr:.4f}")
    print(f"  MAE:       {mae_lr:.4f}")
    
    # Interpretation
    if lr_age_model.coef_[0] < 0:
        print(f"\n✓ Negative slope: Performance decreases by {abs(lr_age_model.coef_[0]):.4f} per year of age")
    else:
        print(f"\n✓ Positive slope: Performance increases by {lr_age_model.coef_[0]:.4f} per year of age")
else:
    print("Performance_Score not available")

## 3.5 Polynomial Regression: Capturing Non-Linear Patterns

In [ ]:
# Test polynomial degrees 2-4
if 'Performance_Score' in age_df_clean.columns:
    print("=" * 70)
    print("POLYNOMIAL REGRESSION COMPARISON")
    print("=" * 70)
    
    poly_results = []
    poly_models = {}
    
    for degree in [2, 3, 4]:
        # Create polynomial features
        poly = PolynomialFeatures(degree=degree, include_bias=False)
        X_train_poly = poly.fit_transform(X_train_age)
        X_test_poly = poly.transform(X_test_age)
        
        # Train model
        poly_model = LinearRegression()
        poly_model.fit(X_train_poly, y_train_perf)
        y_pred_poly = poly_model.predict(X_test_poly)
        
        # Evaluate
        r2 = r2_score(y_test_perf, y_pred_poly)
        rmse = np.sqrt(mean_squared_error(y_test_perf, y_pred_poly))
        mae = mean_absolute_error(y_test_perf, y_pred_poly)
        
        poly_results.append({
            'Degree': degree,
            'R²': r2,
            'RMSE': rmse,
            'MAE': mae
        })
        
        poly_models[degree] = (poly, poly_model)
    
    poly_df = pd.DataFrame(poly_results)
    print(poly_df.to_string(index=False))
    
    # Best polynomial degree
    best_degree = poly_df.loc[poly_df['R²'].idxmax(), 'Degree']
    print(f"\n✓ Best polynomial degree: {int(best_degree)} (highest R²)")
else:
    print("Performance_Score not available")

In [ ]:
# Visualize linear vs polynomial fits
if 'Performance_Score' in age_df_clean.columns:
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))
    
    # Create smooth line for predictions
    age_range = np.linspace(X_age.min(), X_age.max(), 100).reshape(-1, 1)
    
    # Plot 1: Linear regression
    axes[0].scatter(X_test_age, y_test_perf, alpha=0.4, s=30, label='Actual')
    axes[0].plot(age_range, lr_age_model.predict(age_range), 
                'r-', linewidth=2, label=f'Linear Fit (R²={r2_lr:.3f})')
    axes[0].set_xlabel('Age', fontsize=12)
    axes[0].set_ylabel('Performance Score', fontsize=12)
    axes[0].set_title('Linear Regression', fontsize=14, fontweight='bold')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    # Plot 2: Degree 2 polynomial
    poly2, model2 = poly_models[2]
    age_range_poly2 = poly2.transform(age_range)
    r2_poly2 = poly_df[poly_df['Degree'] == 2]['R²'].values[0]
    
    axes[1].scatter(X_test_age, y_test_perf, alpha=0.4, s=30, label='Actual')
    axes[1].plot(age_range, model2.predict(age_range_poly2), 
                'g-', linewidth=2, label=f'Quadratic Fit (R²={r2_poly2:.3f})')
    axes[1].set_xlabel('Age', fontsize=12)
    axes[1].set_ylabel('Performance Score', fontsize=12)
    axes[1].set_title('Polynomial Regression (Degree 2)', fontsize=14, fontweight='bold')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
    # Plot 3: Best polynomial
    best_poly, best_model = poly_models[int(best_degree)]
    age_range_poly_best = best_poly.transform(age_range)
    r2_best = poly_df[poly_df['Degree'] == best_degree]['R²'].values[0]
    
    axes[2].scatter(X_test_age, y_test_perf, alpha=0.4, s=30, label='Actual')
    axes[2].plot(age_range, best_model.predict(age_range_poly_best), 
                'purple', linewidth=2, label=f'Degree {int(best_degree)} (R²={r2_best:.3f})')
    axes[2].set_xlabel('Age', fontsize=12)
    axes[2].set_ylabel('Performance Score', fontsize=12)
    axes[2].set_title(f'Best Polynomial (Degree {int(best_degree)})', 
                     fontsize=14, fontweight='bold')
    axes[2].legend()
    axes[2].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

## 3.6 Identify Peak Performance Age

In [ ]:
# Find age that maximizes predicted performance (for quadratic model)
if 'Performance_Score' in age_df_clean.columns:
    poly2, model2 = poly_models[2]
    
    # Generate predictions for all ages
    all_ages = np.arange(16, 41).reshape(-1, 1)
    all_ages_poly = poly2.transform(all_ages)
    predictions = model2.predict(all_ages_poly)
    
    # Find peak
    peak_idx = predictions.argmax()
    peak_age = all_ages[peak_idx][0]
    peak_performance = predictions[peak_idx]
    
    print("=" * 70)
    print("PEAK PERFORMANCE AGE ANALYSIS (Quadratic Model)")
    print("=" * 70)
    print(f"Peak Performance Age: {peak_age} years")
    print(f"Predicted Performance at Peak: {peak_performance:.4f}")
    
    # Performance at different life stages
    ages_of_interest = [20, 25, 30, 35]
    print(f"\nPredicted Performance by Age:")
    for age in ages_of_interest:
        age_poly = poly2.transform([[age]])
        perf = model2.predict(age_poly)[0]
        pct_of_peak = (perf / peak_performance) * 100
        print(f"  Age {age}: {perf:.4f} ({pct_of_peak:.1f}% of peak)")
    
    # Visualize peak
    plt.figure(figsize=(10, 6))
    plt.plot(all_ages, predictions, 'b-', linewidth=2, label='Predicted Performance')
    plt.scatter([peak_age], [peak_performance], s=200, c='red', 
               marker='*', zorder=5, label=f'Peak Age ({peak_age} years)')
    plt.axvline(x=peak_age, color='red', linestyle='--', alpha=0.5)
    plt.xlabel('Age (years)', fontsize=12)
    plt.ylabel('Predicted Performance Score', fontsize=12)
    plt.title('Performance Trajectory Across Age', fontsize=14, fontweight='bold')
    plt.legend(fontsize=11)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

## 3.7 Age Group Comparison (ANOVA)

In [ ]:
# Create age groups
if 'Performance_Score' in age_df_clean.columns:
    def categorize_age(age):
        if age < 23:
            return 'Young (<23)'
        elif age <= 29:
            return 'Prime (23-29)'
        else:
            return 'Experienced (30+)'
    
    age_df_clean['Age_Group'] = age_df_clean['Age_Years'].apply(categorize_age)
    
    # Group statistics
    print("=" * 70)
    print("PERFORMANCE BY AGE GROUP")
    print("=" * 70)
    group_stats = age_df_clean.groupby('Age_Group')['Performance_Score'].agg([
        ('Count', 'count'),
        ('Mean', 'mean'),
        ('Std', 'std'),
        ('Median', 'median'),
        ('Min', 'min'),
        ('Max', 'max')
    ]).round(4)
    print(group_stats)
    
    # ANOVA test
    young = age_df_clean[age_df_clean['Age_Group'] == 'Young (<23)']['Performance_Score']
    prime = age_df_clean[age_df_clean['Age_Group'] == 'Prime (23-29)']['Performance_Score']
    experienced = age_df_clean[age_df_clean['Age_Group'] == 'Experienced (30+)']['Performance_Score']
    
    f_stat, p_value = stats.f_oneway(young, prime, experienced)
    
    print(f"\n{'='*70}")
    print("ANOVA TEST RESULTS")
    print("="*70)
    print(f"F-statistic: {f_stat:.4f}")
    print(f"P-value: {p_value:.6f}")
    
    if p_value < 0.05:
        print("\n✓ SIGNIFICANT difference between age groups (p < 0.05)")
        print("  Age groups show statistically different performance levels")
    else:
        print("\n✗ NO significant difference between age groups (p >= 0.05)")
        print("  Cannot confidently conclude age groups differ in performance")
    
    # Visualize
    plt.figure(figsize=(10, 6))
    sns.boxplot(data=age_df_clean, x='Age_Group', y='Performance_Score',
               order=['Young (<23)', 'Prime (23-29)', 'Experienced (30+)'])
    plt.title('Performance Distribution by Age Group', fontsize=14, fontweight='bold')
    plt.xlabel('Age Group', fontsize=12)
    plt.ylabel('Performance Score', fontsize=12)
    plt.grid(axis='y', alpha=0.3)
    plt.tight_layout()
    plt.show()

---
# Summary and Key Findings

In [ ]:
print("="*80)
print(" " * 20 + "REGRESSION ANALYSIS - COMPREHENSIVE SUMMARY")
print("="*80)

print("\n" + "="*80)
print("QUESTION 1: POSSESSION VS MATCH OUTCOMES")
print("="*80)
print(f"Binary Classification (Win vs Not-Win):")
print(f"  • Accuracy: {accuracy:.2%}")
print(f"  • ROC-AUC: {roc_auc:.4f}" if 'roc_auc' in locals() else "  • ROC-AUC: N/A")
print(f"  • Coefficient: {log_reg_binary.coef_[0][0]:.4f}")
print(f"\nMulticlass Classification (Win/Draw/Loss):")
print(f"  • Accuracy: {accuracy_multi:.2%}")
print(f"\nConclusion: Possession shows {'WEAK' if accuracy < 0.60 else 'MODERATE' if accuracy < 0.75 else 'STRONG'} predictive power")

print("\n" + "="*80)
print("QUESTION 2: SALARY VS PERFORMANCE")
print("="*80)
best_model_name = models_comparison.loc[models_comparison['R²'].idxmax(), 'Model']
best_r2 = models_comparison['R²'].max()
print(f"Best Model: {best_model_name}")
print(f"  • R² Score: {best_r2:.4f} ({best_r2*100:.1f}% variance explained)")
print(f"  • Performance explains {best_r2*100:.1f}% of salary variance")
print(f"  • {100-best_r2*100:.1f}% of salary determined by other factors")
print(f"\nMost Important Features:")
top_features = abs_coefs.head(3)
for idx, row in top_features.iterrows():
    print(f"  • {row['Feature']}: {row['Avg_Abs_Coef']:.4f}")

if 'Performance_Score' in age_df_clean.columns:
    print("\n" + "="*80)
    print("QUESTION 3: AGE VS PERFORMANCE")
    print("="*80)
    print(f"Linear Regression:")
    print(f"  • R² Score: {r2_lr:.4f}")
    print(f"  • Coefficient: {lr_age_model.coef_[0]:.6f} (performance/year)")
    print(f"\nBest Polynomial Model (Degree {int(best_degree)}):")
    print(f"  • R² Score: {r2_best:.4f}")
    print(f"  • Peak Performance Age: {peak_age} years")
    print(f"\nAge Group Differences:")
    print(f"  • ANOVA p-value: {p_value:.6f}")
    print(f"  • Result: {'SIGNIFICANT' if p_value < 0.05 else 'NOT SIGNIFICANT'}")

print("\n" + "="*80)
print("OVERALL CONCLUSIONS")
print("="*80)
print("1. Possession has limited predictive power for match outcomes")
print("2. Performance metrics explain only a portion of player salaries")
print("3. Age shows a non-linear relationship with performance (inverted-U)")
print("4. Multiple factors beyond simple metrics influence football success")
print("\nRecommendation: Use regression models as ONE tool in a comprehensive")
print("                analysis framework, not as the sole decision-making basis")
print("="*80)